## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-15-41-29 +0000,nyt,Zuckerberg’s Compound Had Something That Viola...,https://www.nytimes.com/2025/08/10/us/zuckerbe...
1,2025-08-10-15-36-59 +0000,startribune,Neal: Sam Howell quiets the alarms calling for...,https://www.startribune.com/minnesota-vikings-...
2,2025-08-10-15-35-23 +0000,nyt,"In Election Cases, Supreme Court Keeps Removin...",https://www.nytimes.com/2025/08/10/us/supreme-...
3,2025-08-10-15-30-00 +0000,startribune,‘Less vitriolic’ than Minneapolis: What to kno...,https://www.startribune.com/less-vitriolic-tha...
4,2025-08-10-15-26-38 +0000,nyt,Heavy Rains Lead to Widespread Flooding in Wis...,https://www.nytimes.com/2025/08/10/us/wisconsi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,26
57,ukraine,16
64,putin,14
338,california,10
15,new,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
162,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...,88
17,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...,73
8,2025-08-10-15-03-01 +0000,nyt,Ukraine and Europe Project United Front Ahead ...,https://www.nytimes.com/2025/08/10/world/europ...,72
173,2025-08-09-16-14-54 +0000,bbc,Putin gives Trump envoy award for CIA official...,https://www.bbc.com/news/articles/c04rd19e5epo...,67
30,2025-08-10-12-17-48 +0000,nyt,"With Ukraine in the Balance, Trump and Putin H...",https://www.nytimes.com/2025/08/10/world/europ...,67


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
162,88,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...
118,32,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
14,30,2025-08-10-14-32-24 +0000,nypost,UK police arrest nearly 500 at protest for ban...,https://nypost.com/2025/08/10/world-news/uk-po...
135,29,2025-08-09-22-47-00 +0000,wsj,Adriana Kugler hasn’t said why she is leaving ...,https://www.wsj.com/economy/central-banking/su...
43,28,2025-08-10-10-00-00 +0000,latimes,"California took center stage in ICE raids, but...",https://www.latimes.com/california/story/2025-...
70,26,2025-08-10-09-30-00 +0000,startribune,Digital assault that shut down St. Paul servic...,https://www.startribune.com/cyberattack-st-pau...
102,25,2025-08-10-06-02-33 +0000,nypost,TikTok influencer Emilie Kiser’s husband alleg...,https://nypost.com/2025/08/10/us-news/influenc...
169,24,2025-08-09-16-40-45 +0000,nypost,Ambassador Mike Huckabee slams British PM Keir...,https://nypost.com/2025/08/09/us-news/ambassad...
65,24,2025-08-10-10-00-00 +0000,latimes,Fast-forward 25 years: Sharing hopes and dream...,https://www.latimes.com/california/story/2025-...
157,24,2025-08-09-19-18-00 +0000,nyt,Zelensky Rejects Ceding Territory to Russia Af...,https://www.nytimes.com/2025/08/09/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
